In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install transformers==3.02

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import rc
from pylab import rcParams
import matplotlib.pyplot as plt
from textwrap import wrap
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_curve,auc


import transformers
from transformers import BertModel, BertTokenizerFast
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import DistilBertModel,DistilBertTokenizer
from transformers import AutoTokenizer,AutoModel,AutoModelWithLMHead
import torch
from torch import nn,optim
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn  
import torch.nn.functional as F  
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu');
import time
from sklearn.metrics import classification_report

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid',palette='muted',font_scale=1.2)
color_palette=['#01BEFE','#FFDD00','#FF7D00','#FF006D','#ADFF02','#8F00FF']
sns.set_palette(sns.color_palette(color_palette))

rcParams['figure.figsize']= 12,6

import warnings
warnings.filterwarnings('ignore')

seed=42
np.random.seed(seed)
torch.manual_seed(seed)



In [7]:
df=pd.read_csv('/content/drive/MyDrive/upwork/wesam/dataset/saadny_data.csv')

In [8]:
df.shape

(3242, 2)

In [9]:
df.head()

,text,labels
0,تحليل السيسي الرقم الصعب فى مؤتمر العشرين ا...,0
1,طالب توجيهي يكتشف حبه لمتابعة النشرة الاقتصادي...,0
2,عانس تجد عريسا رغم بلوغها سن الرابعة والعشرين...,1
3,التنمية لم نضبط متسولين بأول أيام رمضان با...,1
4,شاب ينزعج من خبر تفجير رغم وقوعه كالمعتاد في ب...,0


In [10]:
from sklearn.utils import shuffle
df = shuffle(df)

In [11]:
class_names=['0','1']

In [21]:
#We use pretrained model for bert arabic
Pre_trained_model='asafaya/bert-base-arabic' #ok



In [22]:
tokenizer=AutoTokenizer.from_pretrained(Pre_trained_model);
#tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")

Downloading:   0%|          | 0.00/485 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

In [23]:
class reviews_India_Dataset(Dataset):

  def __init__(self,reviews,targets,tokenizer,max_length):
    self.reviews=reviews
    self.targets=targets
    self.tokenizer=tokenizer
    self.max_length=max_length

  def __len__(self):
    return len(self.reviews)

  def __getitem__(self,item):
    review = str(self.reviews[item])
    targets = self.targets[item]

    encoding = self.tokenizer.encode_plus(
        review,
        max_length=Max_length,
        add_special_tokens=True,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors='pt'
       )
    return {
        'review_text':review,
        'input_ids':encoding['input_ids'].flatten(),
        'attention_mask':encoding['attention_mask'].flatten(),
        'targets' : torch.tensor(targets,dtype=torch.long)
    }

In [24]:
df_train,df_test=train_test_split(df, test_size=0.1, random_state=150)
df_valid,df_test = train_test_split(df_test,test_size=0.8,random_state=150)

print('Print the shape of datasets...')
print(f'Training dataset : {df_train.shape}')
print(f'Testing dataset : {df_test.shape}')
print(f'Validation dataset : {df_valid.shape}')

Print the shape of datasets...
Training dataset : (2917, 2)
Testing dataset : (260, 2)
Validation dataset : (65, 2)


In [25]:
Max_length=100

In [26]:
batch_size=32
def data_loader(df, tokenizer, max_length, batch):
  ds=reviews_India_Dataset(
      reviews=df.text.to_numpy(),
      targets=df.labels.to_numpy(),
      tokenizer=tokenizer,
      max_length=Max_length
  )

  return DataLoader(
      ds,
      batch_size=batch_size,
      num_workers=4
  )

# Load datasets
train_DataLoader=data_loader(df_train,tokenizer,Max_length,batch_size)
test_DataLoader=data_loader(df_test,tokenizer,Max_length,batch_size)
valid_DataLoader=data_loader(df_valid,tokenizer,Max_length,batch_size)

In [27]:
data=next(iter(train_DataLoader))
data.keys()

dict_keys(['review_text', 'input_ids', 'attention_mask', 'targets'])

In [28]:
print('Shape of the data keys...')
print(f"Input_ids : {data['input_ids'].shape}")
print(f"Attention_mask : {data['attention_mask'].shape}")
print(f"targets : {data['targets'].shape}")

Shape of the data keys...
Input_ids : torch.Size([32, 100])
Attention_mask : torch.Size([32, 100])
targets : torch.Size([32])


In [30]:
bert_model = AutoModel.from_pretrained(Pre_trained_model)
#bert_model = AutoModelWithLMHead.from_pretrained("akhooli/gpt2-small-arabic")

In [ ]:
import NER
import RFC
NER=Ner()
RFC=RFC()

In [ ]:
class SentimentClassifier(nn.Module):
  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = AutoModel.from_pretrained(Pre_trained_model)
    self.NER=NER()
    self.RFC=RFC()
    
    #self.bert = AutoModelWithLMHead.from_pretrained("akhooli/gpt2-small-arabic")
    self.drop = nn.Dropout(p=0.5)
    self.linear1 = nn.Linear(768, 512)
    #self.linear1 = nn.Linear(128, 64)
    #without RFC
    #self.linear2=nn.Linear(512,1)
    #without NER
    self.linear3=nn.Linear(512,1)
    self.sigmoid = nn.Sigmoid()
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
model = SentimentClassifier(len(class_names))         #Create an instance / object
model = model.to(device)

In [ ]:
input_ids = data['input_ids'].to(device)
attention_mask = data['attention_mask'].to(device)
print(input_ids.shape)      # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([32, 100])
torch.Size([32, 100])


In [ ]:
F.softmax(model(input_ids,attention_mask), dim=1)

tensor([[0.3790, 0.6210],
        [0.4059, 0.5941],
        [0.3869, 0.6131],
        [0.5368, 0.4632],
        [0.3608, 0.6392],
        [0.5681, 0.4319],
        [0.4365, 0.5635],
        [0.5927, 0.4073],
        [0.6703, 0.3297],
        [0.3984, 0.6016],
        [0.2525, 0.7475],
        [0.5132, 0.4868],
        [0.4840, 0.5160],
        [0.3716, 0.6284],
        [0.4288, 0.5712],
        [0.3573, 0.6427],
        [0.4831, 0.5169],
        [0.5184, 0.4816],
        [0.5027, 0.4973],
        [0.4555, 0.5445],
        [0.1877, 0.8123],
        [0.5154, 0.4846],
        [0.4677, 0.5323],
        [0.5728, 0.4272],
        [0.3696, 0.6304],
        [0.5310, 0.4690],
        [0.2095, 0.7905],
        [0.5136, 0.4864],
        [0.4618, 0.5382],
        [0.3450, 0.6550],
        [0.3656, 0.6344],
        [0.4888, 0.5112]], grad_fn=<SoftmaxBackward0>)

In [ ]:
epochs=5
optimizer=AdamW(model.parameters(),lr=2e-5,correct_bias=False)
total_steps=len(train_DataLoader)*epochs

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn=nn.CrossEntropyLoss().to(device)

In [ ]:
def train(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_observations
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    #Feed data to BERT model
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)     # Clip gradients to avoid exploding gradient problem
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_observations, np.mean(losses)

In [ ]:
def eval_model(model, data_loader,device,loss_fn, n_observations):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      # Feed data to BERT model
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, targets)
      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())
  return correct_predictions.double() / n_observations, np.mean(losses)

In [ ]:
%%time
history = defaultdict(list)
best_accuracy = 0
for epoch in range(epochs):
  print(f'Epoch {epoch + 1}/{epochs}')
  print('-' * 10)
  train_acc, train_loss = train(
    model,
    train_DataLoader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')
  val_acc, val_loss = eval_model(
    model,
    valid_DataLoader,
    device,
    loss_fn,
    len(df_valid)
  )
  print(f'Validation  loss {val_loss} accuracy {val_acc}')
  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/5
----------
Train loss 0.7431281461663868 accuracy 0.49948577305450803
Validation  loss 0.6500752568244934 accuracy 0.5076923076923077

Epoch 2/5
----------
Train loss 0.7307437625916108 accuracy 0.49331504970860474
Validation  loss 0.6676342089970907 accuracy 0.5076923076923077

Epoch 3/5
----------


In [ ]:
test_acc, _ = eval_model(
  model,
  test_DataLoader,
  device,
  loss_fn,
  len(df_test)
)

In [ ]:
test_acc.item()

0.7863805970149254

In [ ]:
def get_predictions(model, data_loader):
  model = model.eval()
  review_texts = []
  predictions = []
  prediction_probs = []
  real_values = []

  with torch.no_grad():
    for d in data_loader:
      texts = d["review_text"]
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      _, preds = torch.max(outputs, dim=1)
      review_texts.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(targets)

  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(
  model,
  test_DataLoader
)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

              precision    recall  f1-score   support

           0       0.80      0.79      0.79       558
           1       0.77      0.78      0.78       514

    accuracy                           0.79      1072
   macro avg       0.79      0.79      0.79      1072
weighted avg       0.79      0.79      0.79      1072



In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc,f1_score,accuracy_score

In [ ]:
f1_score(y_test,y_pred,average='micro')

0.7863805970149255

In [ ]:
accuracy_score(y_test,y_pred)

0.695054945054945

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

In [ ]:
auc_model = auc(fpr, tpr)

In [ ]:
auc_model

0.615846994535519

In [ ]:
#import matplotlib.pyplot as plt
##plt.figure()
#lw = 2
#plt.plot(fpr, tpr, color='darkorange')
#plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
#plt.xlim([0.0, 1.0])
#plt.ylim([0.0, 1.05])
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.title('Receiver operating characteristic example')
#plt.legend(loc="lower right")
#plt.show()

In [ ]:
#false_rate=pd.DataFrame([fpr])
##true_rate=pd.DataFrame([tpr])
#auc_rate=pd.DataFrame([auc])
#false_rate.to_csv("/content/drive/MyDrive/upwork/wesam/dataset/aranews/jointbert_fpr.csv",index=False)
#true_rate.to_csv("/content/drive/MyDrive/upwork/wesam/dataset/aranews/jointbert_tpr.csv",index=False)
#auc_rate.to_csv("/content/drive/MyDrive/upwork/wesam/dataset/aranews/jointbert_auc.csv",index=False)